<a href="https://colab.research.google.com/github/otanet/search_engine_improvement_202208-/blob/main/%E5%88%86%E3%81%8B%E3%81%A1%E6%9B%B8%E3%81%8D_%E3%82%AB%E3%82%A6%E3%83%B3%E3%83%88%E3%81%A8%E6%8E%A8%E8%AB%96%E6%89%8B%E6%B3%95%E3%81%AE%E6%AF%94%E8%BC%83_202209_ipynb_DRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ここでの目的：分かち書きをするとヒットする場合、この分かち書きの現行のツールでどこまでできるかをしめしたものです。
### ※OSSの検索エンジンに頼らないで、自然言語処理の技術でどこまで対応できるか検索したものです。

### ⇒結論として、人手の辞書登録のメンテナンスが必須となります。
### ※OSSの検索エンジンを導入した場合、人手の辞書登録は半自動となると思います。

#### 以下の2つの事例で考察してみました。
#### 例1) 「すとぷり青春チョコレート」
#### →「すとぷり」と「青春チョコレート」に分けると検索でヒットする。

#### 例2) YOASOBIラブレター
#### →「YOASOBI」と「ラブレター」に分けるとヒットする。

#### ⇒分かち書きのツールは大きく2つに分かれます。
#### 1)「カウントベース」: 統計情報から単語を表現する手法
→Mecab/Juman/Janomeなど

####「カウントベース」とは、統計情報から単語を表現する手法のことで、もう一つは、「推論ベース」というニューラルネットワークの手法があります。

#### カウントベースの手法として、文字や単語の「連なり」の頻度分布N-gramをもとに文を生成するプログラムを考えます。

#### 以下の辞書に固有名詞を追加することで、「すとぷり」を一語とみなすように、対応するようにします。

#### 「ipadic」や「ipadic-neologd」

#### 一方、推論ベースの手法は、対象となる文書をもとに言語モデルが構築されるため、あまり制御が効かない可能性があります。

#### 2) 「推論ベース」
主要な歌詞のサイトの表記には対応が見込める反面、辞書登録とのシステムの設計に考慮する必要があります。

まとめ    
今回、検索キーワードに、スペース区切りのないケースがGA上で多く見受けられました。

例1)「すとぷり青春チョコレート」→「すとぷり」 「青春チョコレート」    
例2)「YOASOBIラブレター」→「YOASOBI」 「ラブレター」    

→この対応ができると、PV数が最も向上すると想定しました。※大文字小文字の対応に比べて。    
今回、分かち書きを通じて、カウント手法と推論手法とを比較考察しました。    
OSSの検索エンジンを導入しない場合と導入した場合とを考えることにもなりました。

カウント手法なら、辞書登録が必要になりますが、別の例でも、その語句で分かち書きが出てきてしまう懸念があります。    
一方、推論手法なら、クロール先の歌詞や音楽の文書を確保することが前提となり、ここでの表記の仕方に左右されます。    
後者はOSSの検索エンジンの導入が考えられます。

これらから、辞書登録とOSSの検索エンジンの導入との検討材料になればと思います。

## まず、→Mecab/Juman/Janomeなどの「カウントベース」の手法から見てゆきましょう。

1) Mecabという形態素解析のツールで試してみます。    
「YOASOBIラブレター」    
「すとぷり青春チョコレート」

In [ ]:
# 文章の場合→うまくゆきます！
! pip install mecab-python3 unidic-lite
import MeCab
print(MeCab.Tagger().parse("JKしか勝たん"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 20.2 MB/s 
     |████████████████████████████████| 47.4 MB 2.3 MB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=6dc9a6dbac1f42bd8995c388c5c44fb4d24eced7df40673113f7363f3d248ba3
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite
JK	JK	JK	JK	名詞-普通名詞-一般			0
しか	シカ	シカ	しか	助詞-副助詞			
勝た	カタ	カツ	勝つ	動詞-一般	五段-タ行	未然形-一般	1
ん	ン	ズ	ず	助動詞	助動詞-ヌ	終止形-撥音便	
EOS



In [ ]:
# 検索キーワードが「YOASOBIラブレター」の場合、単語や語句の場合
! pip install mecab-python3 unidic-lite
import MeCab
print(MeCab.Tagger().parse("YOASOBIラブレター"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
YOASOBI	YOASOBI	YOASOBI	YOASOBI	名詞-普通名詞-一般			0
ラブ	ラブ	ラブ	ラブ-love	名詞-普通名詞-一般			1
レター	レター	レター	レター-letter	名詞-普通名詞-一般			1
EOS



「YOASOBIラブレター」    
⇒結果は「YOASOBI」 「ラブ」 「レター」

→「YOASOBI」と「ラブレター」とにわかれませんでした。
→→この場合、「ラブレター」を一語で辞書登録する必要があります。

In [ ]:
! pip install mecab-python3 unidic-lite
import MeCab
print(MeCab.Tagger().parse("すとぷり青春チョコレート"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
す	ス	デス	です	助動詞	助動詞-デス	終止形-一般	
と	ト	ト	と	助詞-接続助詞			
ぷり	プリ	フリ	振り	名詞-普通名詞-助数詞可能			0,2
青春	セーシュン	セイシュン	青春	名詞-普通名詞-一般			0
チョコレート	チョコレート	チョコレート	チョコレート-chocolate	名詞-普通名詞-一般			3
EOS



「すとぷり青春チョコレート」

⇒結果は「す」 「と」 「ぷり」「青春」 「チョコレート」

→「すとぷり」と「青春チョコレート」とにわかれませんでした。    
→→この場合、「青春チョコレート」を一語で辞書登録する必要があります。
→→→この辞書登録は現実的でしょうか??

2) janomeという形態素解析のツールで試してみます。    
「YOASOBIラブレター」    
「すとぷり青春チョコレート」

In [ ]:
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.3 MB/s 


In [ ]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
tokens = t.tokenize("YOASOBIラブレター")
for token in tokens:
  print(token)

YOASOBI	名詞,固有名詞,組織,*,*,*,YOASOBI,*,*
ラブレター	名詞,一般,*,*,*,*,ラブレター,ラブレター,ラブレター


結果:
「YOASOBIラブレター」
⇒結果は「YOASOBI」 「ラブレター」です。
→うまく、分かち書きされました！

In [ ]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
tokens = t.tokenize("すとぷり青春チョコレート")
for token in tokens:
  print(token)

す	名詞,一般,*,*,*,*,す,ス,ス
と	助詞,並立助詞,*,*,*,*,と,ト,ト
ぷり	名詞,一般,*,*,*,*,ぷり,*,*
青春	名詞,一般,*,*,*,*,青春,セイシュン,セイシュン
チョコレート	名詞,一般,*,*,*,*,チョコレート,チョコレート,チョコレート


「すとぷり青春チョコレート」の結果:
⇒結果は「す」 「と」 「ぷり」 「青春」 「チョコレート」でした。

→うまく、分かち書きされませんでした！
→→辞書登録の必要はありますが、すとぷり青春で一語になってしまう懸念が考えられます！⇒「すとぷり」と「青春」では分かち書きできなくなる可能性があります。

以下は新語に強く人力で辞書登録されているNeologDの例です。
ここではまだ登録されていないので、結果は同じです。

In [ ]:
# install MeCab
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [ ]:
# check path to "ipadic-neologd" 
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


In [ ]:
import MeCab

In [ ]:
m = MeCab.Tagger()
sample_txt = "すとぷり青春チョコレート"
print("Mecab:\n", m.parse(sample_txt))

path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = MeCab.Tagger(path)
print("Mecab ipadic NEologd:\n",m.parse(sample_txt))

Mecab:
 す	助動詞,,,,助動詞-デス,終止形-一般,デス,です,す,ス,っす,ッス,和,"","","","","","",助動,ス,ッス,ス,ッス,"","","",7051537469809637,25653
と	助詞,接続助詞,,,,,ト,と,と,ト,と,ト,和,"","","","","","",接助,ト,ト,ト,ト,"","形容詞%F2@-1,動詞%F1","",7098730570457600,25825
ぷり	副詞,,,,,,プリ,ぷり,ぷり,プリ,ぷり,プリ,和,"","","","","","",相,プリ,プリ,プリ,プリ,"2","","",38089573900886528,138569
青春	名詞,普通名詞,一般,,,,セイシュン,青春,青春,セーシュン,青春,セーシュン,漢,"","","","","","",体,セイシュン,セイシュン,セイシュン,セイシュン,"0","C2","",5449463128793600,19825
チョコレート	名詞,普通名詞,一般,,,,チョコレート,チョコレート-chocolate,チョコレート,チョコレート,チョコレート,チョコレート,外,"","","","","","",体,チョコレート,チョコレート,チョコレート,チョコレート,"3","C1","",6645182024000000,24175
EOS

Mecab ipadic NEologd:
 すと	名詞,一般,*,*,*,*,スト,スト,スト
ぷり	名詞,一般,*,*,*,*,*
青春	名詞,一般,*,*,*,*,青春,セイシュン,セイシュン
チョコレート	名詞,一般,*,*,*,*,チョコレート,チョコレート,チョコレート
EOS



In [ ]:
m = MeCab.Tagger()
sample_txt = "YOASOBIラブレター"
print("Mecab:\n", m.parse(sample_txt))

path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = MeCab.Tagger(path)
print("Mecab ipadic NEologd:\n",m.parse(sample_txt))

Mecab:
 YOASOBI	名詞,普通名詞,一般,,,
ラブ	名詞,普通名詞,一般,,,,ラブ,ラブ-love,ラブ,ラブ,ラブ,ラブ,外,"","","","","","",体,ラブ,ラブ,ラブ,ラブ,"1","C3","",10929703993090560,39762
レター	名詞,普通名詞,一般,,,,レター,レター-letter,レター,レター,レター,レター,外,"","","","","","",体,レター,レター,レター,レター,"1","C1","",11184790657180160,40690
EOS

Mecab ipadic NEologd:
 YOASOBI	名詞,固有名詞,組織,*,*,*,*
ラブレター	名詞,一般,*,*,*,*,ラブレター,ラブレター,ラブレター
EOS



## 次に、→ニューラルネットワークの「推論ベース」の手法で試してみます。
→ここでの辞書はWikipediaのサイトをクロール先としております。

In [ ]:
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 20.6 MB/s 
     |████████████████████████████████| 486 kB 39.6 MB/s 
     |████████████████████████████████| 13.4 MB 36.1 MB/s 
     |████████████████████████████████| 120 kB 36.2 MB/s 
     |████████████████████████████████| 880 kB 41.4 MB/s 
     |████████████████████████████████| 6.6 MB 41.6 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=beb451d9f31be82849a19ee5dce761ead8f0a6ce17747b445f208f950c837c48
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d1a7b1de5d22f57c54cbc53320e477c1af9dc98b1bd3bcf5ab0080d8108c8d39
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built ipadic

In [ ]:
import torch
from transformers import BertJapaneseTokenizer, BertModel

In [ ]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [ ]:
# 文章での分かち書き
tokenizer.tokenize('明日は自然言語処理の勉強をしよう。')

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [ ]:
# 検索キーワードでの分かち書き　その1
tokenizer.tokenize('すとぷり青春チョコレート')

['す', 'と', 'ぷ', '##り', '青春', 'チョコレート']

In [ ]:
# 検索キーワードでの分かち書き　その2
tokenizer.tokenize('YOASOBIラブレター')

['YO', '##AS', '##O', '##BI', 'ラブ', '##レ', '##ター']

## ここでは、やはり固有名詞のWebサイトや歌詞サイトのクロールが必要になります。
## OSSのElasticsearchなどを用いるのが前提となり、辞書登録する手間が、音楽サイトやそれらの文書をクロールすることで、人力での辞書登録は半自動化となります。

まとめ    
今回、検索キーワードに、スペース区切りのないケースがGA上で多く見受けられました。

例1)「すとぷり青春チョコレート」→「すとぷり」 「青春チョコレート」    
例2)「YOASOBIラブレター」→「YOASOBI」 「ラブレター」    

→この対応ができると、PV数が最も向上すると想定しました。※大文字小文字の対応に比べて。    
今回、分かち書きを通じて、カウント手法と推論手法とを比較考察しました。    
OSSの検索エンジンを導入しない場合と導入した場合とを考えることにもなりました。

カウント手法なら、辞書登録が必要になりますが、別の例でも、その語句で分かち書きが出てきてしまう懸念があります。    
一方、推論手法なら、クロール先の歌詞や音楽の文書を確保することが前提となり、ここでの表記の仕方に左右されます。    
後者はOSSの検索エンジンの導入が考えられます。

これらから、辞書登録とOSSの検索エンジンの導入との検討材料になればと思います。